In [1]:
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.metrics import root_mean_squared_error
import plotly.express as px

bpac3 = yf.Ticker("BPAC3.SA")
df_bpac3 = bpac3.history(start="2021-01-01",end="2023-12-30")

total_rows = len(df_bpac3)
legs = 21

df_train = df_bpac3.iloc[:total_rows - legs]
df_test = df_bpac3.iloc[total_rows - legs:]

training_set = df_train['Close'].values

In [2]:
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set.reshape(-1,1))

intervalo_entre_os_dias = 3 # Dias vistos para trás

X_train = []
y_train = []

for i in range(intervalo_entre_os_dias, len(training_set)):
  X_train.append(training_set_scaled[i-intervalo_entre_os_dias:i, 0]) #vai pegar sempre os 60 dias para tras
  y_train.append(training_set_scaled[i, 0]) #vai pegar sempre todos os valores do intervalo

X_train, y_train = np.array(X_train), np.array(y_train)

batch_size = len(X_train) # Quantidade de registros que passará para a rede
time_steps = intervalo_entre_os_dias # Quantidade de dias que serão usados para prever o próximo
features = 1 

X_train = np.reshape(X_train, (batch_size, time_steps, features)) # Transforma o array para ter 3 dimensões 
     

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

rnn = Sequential()

rnn.add(LSTM(units=50, return_sequences=True, input_shape=(time_steps, features)))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50))
rnn.add(Dropout(0.5))

rnn.add(Dense(units=1))

In [ ]:
relacao = {'epochs':[], 'rmse':[]}
for epoch in range(100, 2001, 100):
    rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    '''
    Epochs -> Quantidade de vezes que o modelo vai treinar
    '''
    rnn.fit(X_train, y_train, epochs = epoch, batch_size = 32)

    fechamento_real = df_test['Close'].values

    dataset_total = pd.concat((df_bpac3['Close'], df_test['Close']), axis=0)
    inputs = dataset_total[len(dataset_total) - len(df_test) - intervalo_entre_os_dias:].values
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)

    #X_test = []
    X_test = [inputs[i-intervalo_entre_os_dias:i, 0] for i in range(intervalo_entre_os_dias, len(inputs))]
    # for i in range(intervalo_entre_os_dias, len(inputs)):
    #   X_test.append(inputs[i-intervalo_entre_os_dias:i, 0])

    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    predict = rnn.predict(X_test)

    variacao_final = sc.inverse_transform(predict)
    test = df_test['Close'].values
    relacao['rmse'].append(root_mean_squared_error(test, variacao_final))
    relacao['epochs'].append(epoch)
    print(f'Finalizou época {epoch}')
    print(relacao)


In [ ]:
print("Valores de épocas e rmse:")
print(relacao)

In [ ]:


df = pd.DataFrame(relacao)

fig = px.line(
    df, 
    x='epochs',
    y=['rmse'],
    
)
fig.update_layout(title="Relação Épocas x RMSE obtido para uma RNN de janela 3 com 5 camadas para ação BPAC3", xaxis_title="Épocas", yaxis_title="RMSE Obtido") 

fig.show()